<h1>Decision Trees Implementation</h1>

In [14]:
import pandas as pd
import numpy as np
import itertools
import cv2
from collections import Counter
from sklearn import datasets
from sklearn.model_selection import train_test_split

<h3>Decision Trees Concept</h3>

Decision trees are a a set of nodes arranged in a parent-child relationship. The nodes of a decision tree has the following structure:

Node {  
    self.feature,  
    self.threshold,  
    self.left,  
    self.right,  
    self.value,  
    self.is_leaf_node  
}

In order for a decision tree to be a true decision tree, there are various functions used to create the tree:

DecisionTree {  
    self.fit(),  
    self.predict(),  
    self._tree_traversal(),  
    self._information_gain(),  
    self._entropy(),  
    self._best_split(),  
    self._split()  
}

For the various functions of a decision tree, there may be additional helper functions to be implemented so as to make the code more organized.


<h3>Node and Decision Tree Class</h3>

In [13]:

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    def is_leaf_node(self):
        return self.value is not None
  
class DecisionTree:
    def __init__(self, min_samples_split = 2, max_depth = 100, n_features = None, split_method = "gini"):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.split_method = split_method
        self.nodes = 0
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X, y)
    
    '''
    _grow_tree(self, X_train, y_train, depth = 0)
    Function recursively calls itself until one of the following conditions is met:
        1. depth is greater than or equal to the max depth
        2. The number of labels left to split are 1
        3. The number of samples are less than the minimum number of samples to make a split
    Each pass performs each of the following actions (if break condition is not met):
        1. Create a list of all possible features that can be split.
        2. Retrieve the best threshold and best feature to split using _best_split().
        3. Grow the tree by building a node with left and right child nodes (recursion occurs here).
    '''
    def _grow_tree(self, X, y, depth = 0):
        print("Depth: {}".format(depth), end="\r")
        n_samples, n_feats = X.shape # [0], X.shape[3]
        n_labels = len(np.unique(y))

        #stopping criteria
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value = leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace = False)

        #find best split
        best_feat, best_thresh = self._best_split(X, y, feat_idxs)

        #create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        self.nodes += 1
        return Node(best_feat, best_thresh, left, right)

    '''
    _best_split(X_train, y_train, feature indicies)
    This function computes the best split by calling the information gain for all feature indicies,
    which uses entropy. This function returns split_threshold (the threshold at which a value is "yes" or "no"
    at a node) and split_idx (the index where the split occurs in the list of features).
    '''
    def _best_split(self, X, y, feat_idxs):
        #should be -1 if using information gain
        if self.split_method == "gini":
            best_gain = 1
            split_idx, split_threshold = None, None

            for feat_idx in feat_idxs:
                X_col = X[:, feat_idx]
                thresholds = np.unique(X_col)

                for thresh in thresholds:
                    #gain = self._information_gain(y, X_col, thresh)
                    gain = self._gini(y)

                    #was gain > best_gain
                    if gain < best_gain:
                        best_gain = gain
                        split_idx = feat_idx
                        split_threshold = thresh
        else:
            best_gain = -1
            split_idx, split_threshold = None, None

            for feat_idx in feat_idxs:
                X_col = X[:, feat_idx]
                thresholds = np.unique(X_col)

                for thresh in thresholds:
                    gain = self._information_gain(y, X_col, thresh)
                    if gain > best_gain:
                        best_gain = gain
                        split_idx = feat_idx
                        split_threshold = thresh

        return split_idx, split_threshold

    def _gini(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return 1 - sum(p**2 for p in ps)

    '''
    _information_gain(y_test, X_col, threshold)
    This function computes the information gain (the higher; the better) by computing the entropy
    of the parent node and the entropies of the potential children nodes. The information gain is calculated
    by the following equation: info_gain = entropy(parent) - (entropy(left_child) + entropy(right_child)).
    '''
    def _information_gain(self, y, X_col, threshold):
        parent_entr = self._entropy(y)
        left_idxs, right_idxs = self._split(X_col, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entr = (n_l/n) * e_l + (n_r/n) * e_r

        info_gain = parent_entr - child_entr
        return info_gain

    def _split(self, X_col, split_thresh):
        left_idxs = np.argwhere(X_col <= split_thresh).flatten()
        right_idxs = np.argwhere(X_col > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p > 0])
            

    def _most_common_label(self, y):
        counter = Counter(y)
        val = counter.most_common(1)[0][0]
        return val
    
    #predict(X_test)
    #returns the value given by the tree after traversing the tree using the rules fit by the training data.
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def number_of_nodes(self):
        return self.nodes

<h3>Setting Up Data</h3>

<h2>Preparing the Dataset</h2>

In [22]:
data = pd.read_csv("..\data\HAM10000_metadata.csv")

# Creating a dataframe with 10%
# values of original dataframe
data = data.sample(frac = 0.08)
testData = data.sample(frac = 0.2)
 
# Creating dataframe with
# rest of the 84% values
trainData = data.drop(testData.index)

# class labels
classes = [['mel', 'Melanoma'], ['nv', 'Melanocytic nevus'], ['bcc', 'Basal cell carcinoma'], 
           ['akiec', 'Actinic keratosis / Bowen’s disease'], ['bkl', 'Benign keratosis'], ['df', 'Dermatofibroma'],
           ['vasc', 'Vascular lesion']]


# row = data.iloc[1]:
testImages = []
testLabels = []
for index, row in testData.iterrows():
    if index % 1000 == 0:
        print (index)
    img = cv2.imread("..\data\\allData\HAM10000_images\\" + row['image_id'] +".jpg", flags= cv2.IMREAD_GRAYSCALE)
    # img = img / 255  ## changes values 0-1
    dim = (108, 81)
    img = cv2.resize(img, dim)  ## resize
    testImages.append(np.reshape(img, 108*81))
    arr = None
    if row['dx'] == 'mel':
        arr =  0
    elif row['dx'] == 'nv':
        arr =  1
    elif row['dx'] == 'bcc':
        arr =  2
    elif row['dx'] == 'akiec':
        arr =  3
    elif row['dx'] == 'bkl':
        arr =  4
    elif row['dx'] == 'df':
        arr =  5
    else:
        arr =  6
    testLabels.append(arr)

trainImages = []
trainLabels = []
for index, row in trainData.iterrows():
    if index % 1000 == 0:
        print (index)
    img = cv2.imread("..\data\\allData\HAM10000_images\\" + row['image_id'] +".jpg", flags= cv2.IMREAD_GRAYSCALE)
    # img = img / 255  ## changes values 0-1
    dim = (108, 81)
    img = cv2.resize(img, dim)  ## resize
    trainImages.append(np.reshape(img, 108*81))
    arr = None
    if row['dx'] == 'mel':
        arr =  0
    elif row['dx'] == 'nv':
        arr =  1
    elif row['dx'] == 'bcc':
        arr =  2
    elif row['dx'] == 'akiec':
        arr =  3
    elif row['dx'] == 'bkl':
        arr =  4
    elif row['dx'] == 'df':
        arr =  5
    else:
        arr =  6
    trainLabels.append(arr)

In [19]:
xTrain = np.array(trainImages)
yTrain = np.array(trainLabels)
xTest = np.array(testImages)
yTest = np.array(testLabels)

<h3>Fitting the Decision Tree</h3>

In [23]:
# clf_tree = DecisionTree()    ## ~62% for both gini and non gini with 100 dpeth 5% of data set train            increasing to 20% data set train got 63% 
clf_tree = DecisionTree(min_samples_split=2, max_depth= 200)  # 61.09% accuracy
clf_tree.fit(xTrain, yTrain)
predictions = clf_tree.predict(xTest)

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

acc = accuracy(yTest, predictions)
print("Accuracy: {}".format(acc))

Accuracy: 0.6109725685785536
